# Ops-Style PDF Tool Demo

This notebook shows how to call the document tools directly like an ops analyst:
1. Load a PDF
2. Build document map (sections, definitions, xrefs)
3. Search for covenants
4. Read spans by anchor
5. Follow references
6. Save findings in scratchpad and reading trail


In [1]:
import sys
from pathlib import Path

# Adjust this path if your notebook cwd is different
repo_root = Path('/Users/infantjerin/Projects/git/coding-agent-doc')
sys.path.insert(0, str(repo_root / 'src'))

from tools.document_tools import (
    LoadDocumentsTool,
    BuildDocMapTool,
    SearchInDocTool,
    ReadSpanTool,
    FollowReferenceTool,
    QuoteEvidenceTool,
    ConsistencyCheckTool,
)
from tools.state_tools import WriteScratchpadTool, ReadScratchpadTool, AppendReadingTrailTool


In [4]:
# Put your real PDF path here
pdf_path = '/Users/infantjerin/Projects/git/coding-agent-doc/resources/AbbieVie Term Loan Credit Agreement.pdf'
# Example fallback to sample txt in repo if no PDF yet:
# pdf_path = str(repo_root / 'examples' / 'sample_credit_agreement.txt')

loader = LoadDocumentsTool()
doc_store = loader.run([pdf_path])
doc_store["documents"][0]["total_pages"]


121

In [5]:
mapper = BuildDocMapTool()
doc_map = mapper.run(doc_store)

summary = {
    'documents': len(doc_map['document_store']['documents']),
    'sections': len(doc_map['sections']),
    'definitions': len(doc_map['definitions']),
    'xrefs': len(doc_map['xrefs']),
}
summary


{'documents': 1, 'sections': 143, 'definitions': 123, 'xrefs': 400}

In [6]:
doc_map['sections']

[{'id': 'doc-0:section:1.01',
  'doc_id': 'doc-0',
  'section_no': '1.01',
  'title': 'Certain Defined Terms. ..................................................................................... 1',
  'level': 2,
  'page_start': 2,
  'page_end': 2,
  'anchor': 'doc-0:p2:b6'},
 {'id': 'doc-0:section:1.02',
  'doc_id': 'doc-0',
  'section_no': '1.02',
  'title': 'Computation of Time Periods. ........................................................................ 27',
  'level': 2,
  'page_start': 2,
  'page_end': 2,
  'anchor': 'doc-0:p2:b7'},
 {'id': 'doc-0:section:1.03',
  'doc_id': 'doc-0',
  'section_no': '1.03',
  'title': 'Accounting Terms. .......................................................................................... 27',
  'level': 2,
  'page_start': 2,
  'page_end': 2,
  'anchor': 'doc-0:p2:b8'},
 {'id': 'doc-0:section:1.04',
  'doc_id': 'doc-0',
  'section_no': '1.04',
  'title': 'Terms Generally. ...................................................................

In [7]:
search = SearchInDocTool()
hits = search.run(doc_map=doc_map, query='covenants', scope='section', top_k=5)
hits


[{'type': 'section',
  'score': 4,
  'anchor': 'doc-0:p2:b41',
  'doc_id': 'doc-0',
  'section_no': '5.01',
  'title': 'Affirmative Covenants. .................................................................................... 52',
  'text': 'Section 5.01. Affirmative Covenants. .................................................................................... 52'},
 {'type': 'section',
  'score': 4,
  'anchor': 'doc-0:p2:b42',
  'doc_id': 'doc-0',
  'section_no': '5.02',
  'title': 'Negative Covenants. ........................................................................................ 59',
  'text': 'Section 5.02. Negative Covenants. ........................................................................................ 59'},
 {'type': 'section',
  'score': 4,
  'anchor': 'doc-0:p56:b23',
  'doc_id': 'doc-0',
  'section_no': '5.01',
  'title': 'Affirmative Covenants.  So long as any Advance shall remain unpaid or any',
  'text': 'Section 5.01. Affirmative Covenants.  So long 

In [ ]:
reader = ReadSpanTool()
trail = AppendReadingTrailTool()
state = {'scratchpad': {}, 'reading_trail': []}

first_anchor = hits[0]['anchor'] if hits else None
span = reader.run(doc_map=doc_map, anchor=first_anchor) if first_anchor else None
if first_anchor:
    trail.run(state=state, anchor=first_anchor)

span


In [ ]:
# Follow one reference from the first anchor (if available)
follower = FollowReferenceTool()
refs = [r for r in doc_map['xrefs'] if first_anchor and r.get('from_anchor') == first_anchor]
refs[:3]


In [ ]:
resolved = None
if refs:
    resolved = follower.run(doc_map=doc_map, ref_id=refs[0]['id'])
    if resolved.get('resolved'):
        trail.run(state=state, anchor=resolved['anchor'])
resolved


In [ ]:
writer = WriteScratchpadTool()
reader_pad = ReadScratchpadTool()

writer.run(state=state, key='covenants_check', content={
    'query': 'covenants',
    'top_hit': hits[0] if hits else None,
    'resolved_ref': resolved,
})

reader_pad.run(state=state, key='covenants_check')


In [ ]:
quote = QuoteEvidenceTool()
check = ConsistencyCheckTool()

anchors = list(dict.fromkeys(state['reading_trail']))
evidence = quote.run(doc_map=doc_map, anchors=anchors)
consistency = check.run(claim='What are the financial covenants?', evidence=evidence)

{'reading_trail': state['reading_trail'], 'evidence': evidence, 'consistency': consistency}
